In [6]:
import random
import copy
from load import *
import load
import networkx as nx
import torch
import matplotlib.pyplot as plt

In [2]:
g=load.load_random_hetero_graph('../data/PAAD_network', Specified_quantity={'lncrna': 1500, 'mirna': 168, 'mrna': 2519})
adjacency_matrix=torch.from_numpy(np.array(nx.adjacency_matrix(dgl.to_homogeneous(g).to_networkx().to_undirected()).todense(),dtype=float))
gragh = nx.Graph()
for i in range(len(adjacency_matrix)):
    gragh.add_node(i)
    gragh.nodes[i]["data"] = i

for i in range(len(adjacency_matrix)):
    for j in range(len(adjacency_matrix)):
        if adjacency_matrix[i,j]:
            gragh.add_edge(i,j)


Graph(num_nodes={'lncrna': 1500, 'mirna': 168, 'mrna': 2519},
      num_edges={('lncrna', 'lnc_lnc', 'lncrna'): 12654, ('lncrna', 'lnc_mi', 'mirna'): 186, ('lncrna', 'm_lnc', 'mrna'): 23276, ('mirna', 'mi_lnc', 'lncrna'): 186, ('mirna', 'mi_m', 'mrna'): 17320, ('mirna', 'mi_mi', 'mirna'): 132, ('mrna', 'lnc_m', 'lncrna'): 23276, ('mrna', 'm_m', 'mrna'): 60230, ('mrna', 'm_mi', 'mirna'): 17320},
      metagraph=[('lncrna', 'lncrna', 'lnc_lnc'), ('lncrna', 'mirna', 'lnc_mi'), ('lncrna', 'mrna', 'm_lnc'), ('mirna', 'lncrna', 'mi_lnc'), ('mirna', 'mrna', 'mi_m'), ('mirna', 'mirna', 'mi_mi'), ('mrna', 'lncrna', 'lnc_m'), ('mrna', 'mrna', 'm_m'), ('mrna', 'mirna', 'm_mi')])


In [3]:
#画图
# plt.subplot(211)
# nx.draw_networkx(gragh, with_labels=True,node_size=50,font_size=8)
# plt.show()

In [4]:
#去掉真实网络中去点度为0的孤立节点
new_nodes = [_ for _, v in nx.degree(gragh) if v != 0]

# plt.subplot(211)
graph_new = gragh.subgraph(new_nodes)
# nx.draw_networkx(graph_new, with_labels=True,node_size=50,font_size=8)
# plt.show()


In [5]:
print(len(new_nodes))

3217


In [60]:
#对联通子图进行判断
# N = 50
# N = 100
# N = 150
# N = 200
# N = 250
# N = 300
# # N = 400
# idex = 12
# CS = []
# CS_len = []
# init_graph = []
# for c in nx.connected_components(graph_new):
#     # 得到不连通的子集
#     nodeSet = list(graph_new.subgraph(c).nodes())
#     CS.append(c)
#     CS_len.append(len(nodeSet))
#     if len(nodeSet) <N:
#         init_graph.append(nodeSet)
# print(CS_len)
# graph_new = graph_new.subgraph(CS[0])
# #小于N的，直接算子图
#大于N的，直接进行采样

[3164, 2, 2, 2, 3, 2, 3, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2]


In [7]:
print(graph_new.nodes())
#获取新图的节点
new_sort_nodes = list(graph_new.nodes())
node_dict = {}
node_dict_reverse = {}
#记录新节点在邻接矩阵中的index下标
for i,n in enumerate(new_sort_nodes):
    node_dict[n] = i
    node_dict_reverse[i] = n
new_A=np.array(nx.adjacency_matrix(graph_new).todense())
print(node_dict_reverse)
torch.save(node_dict_reverse,"node_mapping_3217.pt")
#改变邻接矩阵
print(new_A)

[0, 2, 4, 5, 6, 7, 8, 9, 11, 12, 16, 19, 20, 22, 23, 24, 30, 31, 32, 34, 36, 38, 39, 40, 41, 45, 46, 52, 53, 54, 55, 59, 60, 63, 64, 65, 66, 67, 71, 78, 80, 81, 82, 83, 85, 86, 87, 95, 98, 101, 102, 103, 104, 105, 108, 112, 117, 121, 122, 123, 125, 126, 135, 136, 137, 138, 139, 140, 142, 144, 145, 146, 148, 149, 150, 154, 157, 163, 165, 166, 170, 171, 173, 178, 180, 185, 188, 189, 190, 191, 194, 200, 203, 204, 205, 206, 208, 210, 212, 213, 215, 218, 219, 220, 222, 224, 225, 226, 227, 229, 232, 236, 237, 238, 240, 241, 242, 244, 245, 254, 256, 262, 263, 264, 265, 267, 268, 270, 272, 275, 279, 280, 282, 283, 287, 288, 292, 299, 301, 304, 305, 306, 307, 308, 312, 314, 316, 319, 321, 322, 327, 328, 332, 334, 337, 339, 346, 347, 350, 358, 359, 360, 362, 364, 366, 367, 369, 376, 379, 381, 382, 385, 388, 389, 395, 396, 397, 398, 399, 402, 403, 405, 409, 411, 412, 418, 421, 422, 423, 424, 425, 427, 428, 429, 430, 433, 434, 436, 437, 439, 441, 442, 443, 444, 445, 451, 452, 453, 454, 455, 456, 4

In [62]:
print(len(new_A))
print(680 in node_dict.keys())

3164
False


In [63]:
#首先需要获取所有节点的邻居集合
node_neigh = {}
for i in range(len(new_A)):
    for j in range(len(new_A)):
        if new_A[i,j]:
            if node_dict_reverse[i] not in node_neigh.keys():
                node_neigh[node_dict_reverse[i]] = [node_dict_reverse[j]]
            else:
                node_neigh[node_dict_reverse[i]].append(node_dict_reverse[j])
print(node_neigh)

{0: [39, 220, 254, 362, 532, 937, 959, 1003, 1059, 1206, 1242, 1253, 1259, 2034, 2063, 2335, 2458, 2566, 2624, 2892, 2929, 2930, 3018, 3202, 3223, 3243, 3253, 3290, 3755, 3814, 3845, 3889, 4013], 2: [4035], 4: [5, 30, 54, 59, 66, 319, 445, 522, 561, 614, 755, 773, 786, 787, 788, 796, 882, 916, 1015, 1048, 1109, 1128, 1142, 1215, 1230, 1325, 1362, 1375, 1393, 1692, 1827, 1883, 1897, 1913, 2037, 2054, 2067, 2068, 2164, 2196, 2216, 2220, 2271, 2347, 2425, 2427, 2467, 2476, 2623, 2729, 2784, 2815, 2896, 2897, 3010, 3050, 3203, 3326, 3509, 3551, 3579, 3634, 3737, 3847, 3887, 3897, 3944, 3990, 4041, 4154], 5: [4, 59, 319, 522, 561, 755, 796, 921, 1050, 1128, 1140, 1215, 1325, 2054, 2067, 2068, 2220, 2271, 2427, 2467, 2729, 3010, 3509, 3897], 6: [7, 306, 307, 376, 457, 572, 758, 1172, 1311, 1378, 1849, 2591, 2837, 3187, 3577, 3952], 7: [6, 457, 1172, 1311, 1312, 1798, 3154, 3187, 3610, 3636, 3900, 4143], 8: [2882], 9: [4164, 4172], 16: [1108, 1360, 2228, 3612], 19: [545, 602, 1273, 1759, 1849

In [64]:
#需要对把新节点按度排序
degree_list = [(node,v) for node, v in nx.degree(graph_new) ]
nodes_degree_dict = {}
for node,degree in degree_list:
    nodes_degree_dict[node] = degree
# print(degree_list)
# degree_sorted_nodes = sorted(degree_list,key=lambda x: x[1],reverse=True)
degree_sorted_nodes = sorted(degree_list,key=lambda x: x[1])
print(degree_sorted_nodes)

[(2, 1), (8, 1), (23, 1), (34, 1), (46, 1), (112, 1), (117, 1), (125, 1), (139, 1), (204, 1), (210, 1), (218, 1), (229, 1), (240, 1), (270, 1), (301, 1), (305, 1), (332, 1), (346, 1), (364, 1), (402, 1), (422, 1), (428, 1), (442, 1), (443, 1), (456, 1), (469, 1), (512, 1), (513, 1), (533, 1), (550, 1), (552, 1), (574, 1), (616, 1), (621, 1), (669, 1), (676, 1), (696, 1), (772, 1), (775, 1), (783, 1), (790, 1), (812, 1), (822, 1), (895, 1), (925, 1), (933, 1), (942, 1), (957, 1), (984, 1), (1030, 1), (1040, 1), (1047, 1), (1057, 1), (1075, 1), (1080, 1), (1114, 1), (1135, 1), (1154, 1), (1164, 1), (1168, 1), (1171, 1), (1174, 1), (1177, 1), (1188, 1), (1195, 1), (1198, 1), (1201, 1), (1212, 1), (1223, 1), (1236, 1), (1346, 1), (1352, 1), (1413, 1), (1430, 1), (1436, 1), (1445, 1), (1451, 1), (1463, 1), (1476, 1), (1481, 1), (1486, 1), (1490, 1), (1497, 1), (1686, 1), (1703, 1), (1720, 1), (1721, 1), (1741, 1), (1742, 1), (1761, 1), (1764, 1), (1776, 1), (1800, 1), (1807, 1), (1809, 1), 

In [65]:
#写一个函数，输入为list[node,...,],输出为按照度排序的list，相同degree要随机打乱，每次生成结果不同
def get_degree_sorted_list(nodes):
    #按度排序好
    nodes = sorted(nodes,key=lambda x:nodes_degree_dict[x])
    degree_sorted_nodes = [(node,nodes_degree_dict[node]) for node in nodes]
    #相同度要随机打乱
    dict_degree = {}
    #获取各个degree的list集合
    for node,degree in degree_sorted_nodes:
        if degree not in dict_degree.keys():
            dict_degree[degree] = [node]
        else:
            dict_degree[degree].append(node)
    #将各个list随机打乱，然后按度拼接在一起
    for degree,node_list in dict_degree.items():
        random.shuffle(dict_degree[degree])
    # dict_degree_key = sorted(dict_degree.keys(),reverse=True)
    dict_degree_key = sorted(dict_degree.keys())
    sorted_nodes = []
    for key in dict_degree_key:
        sorted_nodes.extend(dict_degree[key])
    return sorted_nodes
    

In [66]:
nodes = [node for node,_ in degree_list]
sorted_nodes = get_degree_sorted_list(nodes)
#将度相同节点随机打乱
# all_nodes_divide_by_degree = []
# dict_degree = {}
# #获取各个degree的list集合
# for node,degree in degree_sorted_nodes:
#     if degree not in dict_degree.keys():
#         dict_degree[degree] = [node]
#     else:
#         dict_degree[degree].append(node)
# #将各个list随机打乱，然后按度拼接在一起
# for degree,node_list in dict_degree.items():
#     random.shuffle(dict_degree[degree])
# dict_degree_key = sorted(dict_degree.keys(),reverse=True)
# sorted_nodes = []
# for key in dict_degree_key:
#     sorted_nodes.extend(dict_degree[key])
# print(len(sorted_nodes))


In [67]:
# print(nodes)
print(sorted_nodes)

[1164, 621, 3060, 3827, 822, 2326, 1040, 2635, 3172, 4064, 332, 1445, 1895, 3089, 533, 812, 696, 1720, 1490, 2247, 2426, 1047, 2823, 1223, 1920, 3362, 957, 1195, 3997, 3283, 402, 4182, 2806, 442, 2282, 1859, 2406, 1198, 3053, 2946, 2974, 1174, 2587, 2660, 3586, 772, 1057, 1236, 2957, 4022, 2840, 2954, 4077, 34, 3397, 3953, 1937, 1481, 3132, 112, 2348, 550, 1030, 513, 3361, 23, 210, 2711, 1742, 2448, 1982, 2153, 1686, 270, 2226, 933, 1080, 4050, 1918, 2987, 942, 3126, 1915, 1346, 783, 1451, 3431, 2641, 2438, 3886, 3480, 1497, 1800, 4107, 3964, 1721, 422, 218, 1741, 2119, 3529, 1430, 2922, 2254, 1945, 240, 2330, 3834, 2688, 552, 984, 4008, 3782, 2312, 3196, 1965, 3642, 669, 3998, 925, 4142, 3497, 1761, 574, 616, 2440, 2674, 3565, 443, 1703, 1075, 2258, 1764, 469, 2095, 1966, 1188, 1809, 1168, 3584, 775, 3389, 46, 2526, 4059, 2952, 1135, 1177, 139, 3528, 3150, 3016, 301, 346, 3924, 1962, 3928, 2244, 1212, 2659, 1114, 1992, 2601, 2680, 1807, 2256, 895, 3224, 229, 428, 1486, 2, 117, 2124, 3

In [68]:
#使用随机游走采样方法
#按照度由大到小的顺序进行采样，直到满足终止条件
#采样终止条件是所有节点都被采样到
# nodes_if_choosed = [0] * len(new_sort_nodes)
# all_subgraph = []
# cursor = 0
# while sum(nodes_if_choosed) != len(new_sort_nodes):
#     sub_graph = set([])
#     node = random.randint(0,len(new_sort_nodes)-1)
#     while node in sub_graph:
#         node = random.randint(0,len(new_sort_nodes)-1)
#     sub_graph.add(node_dict_reverse[node])
#     nodes_if_choosed[node] = 1
#     while len(sub_graph) != N:
# 
#         #随机选一个邻居加入子图中
#         neigh_nodes = node_neigh[node_dict_reverse[node]]
#         node = random.choice(neigh_nodes)
# 
#         sub_graph.add(node)
#         nodes_if_choosed[node_dict[node]] = 1
#         node = node_dict[node]
# 
#     all_subgraph.append(list(sub_graph))
#     print(list(sub_graph))
print(len(sorted_nodes))

3164


In [69]:
#用K阶邻居采样
#N表示每个子图大小
#按照度由大到小的顺序进行采样，直到满足终止条件
#按照度由小到大的顺序进行采样，直到满足终止条件
#采样终止条件是所有节点都被采样到
nodes_if_choosed = [0] * len(sorted_nodes)
all_subgraph = []
cursor = 0
while sum(nodes_if_choosed) != len(sorted_nodes):
    sub_graph = []
    #从new_sort_nodes，随机产生一个节点，生成K阶邻居
    node = degree_sorted_nodes[cursor][0]
    # node = sorted_nodes[cursor]
    cursor+=1
    sub_graph.append(node)
    now_nodes = [node]
    neigh_nodes_k = []
    for n in now_nodes:
        neigh_nodes_k.extend(node_neigh[n])
    nodes_if_choosed[node_dict[node]] = 1
    #开始添加node的K阶邻居，直到数量达到N
    while len(set(sub_graph)) != N:
        #先判断现有节点+新K-阶邻居节点够不够达到N，不够直接全加，否则按度分布加
        nodes_can_add = [node for node in neigh_nodes_k]
        if (len(sub_graph)+len(set(nodes_can_add))) <N:
            for n in nodes_can_add:
                nodes_if_choosed[node_dict[n]] = 1
                sub_graph.append(n)
            # sub_graph.extend(nodes_can_add)
        else:
            #邻居节点按照度排序
            # new_sorted_nodes = get_degree_sorted_list(list(set(nodes_can_add)))
            random.shuffle(nodes_can_add)
            new_sorted_nodes = nodes_can_add.copy()
            print(len(sub_graph))
            for n in new_sorted_nodes:
                nodes_if_choosed[node_dict[n]] = 1
                sub_graph.append(n)
                if len(set(sub_graph)) == N:
                    break
        now_nodes = neigh_nodes_k.copy()
        neigh_nodes_k = []
        for n in now_nodes:
            neigh_nodes_k.extend(node_neigh[n])
        
    all_subgraph.append(list(set(sub_graph)))
    print(list(set(sub_graph)))

16
[4097, 2, 2563, 3074, 2053, 3081, 1546, 3595, 1548, 1551, 3089, 2066, 3603, 2577, 3093, 3096, 1563, 3612, 3100, 1566, 4128, 4131, 2551, 4133, 1574, 2086, 1534, 4139, 2606, 2096, 2610, 3635, 4148, 3639, 2104, 2105, 1595, 4155, 4160, 2113, 2114, 3651, 2628, 2112, 2631, 3655, 4169, 4170, 2123, 4171, 4173, 2638, 3662, 3664, 2640, 1618, 2131, 3147, 3667, 87, 2136, 2647, 2138, 3163, 3164, 1629, 2654, 2144, 2661, 3687, 1640, 1641, 3181, 2162, 3188, 3703, 3709, 3198, 2173, 638, 2181, 1671, 2183, 3209, 2697, 3211, 2700, 3213, 2190, 1679, 1681, 1682, 1683, 2193, 2710, 2199, 2712, 2202, 1691, 1695, 2207, 3745, 2210, 3747, 3236, 2212, 3748, 2213, 3752, 2218, 1706, 1709, 2223, 3762, 3252, 3253, 3254, 2230, 2744, 3258, 2748, 3260, 1730, 1731, 1733, 1734, 1737, 2761, 2769, 2260, 3797, 2776, 1754, 3803, 2268, 1755, 3802, 2271, 2784, 2789, 2278, 3301, 2792, 2793, 3819, 3307, 2803, 1781, 3830, 3326, 3327, 3839, 2817, 2306, 3845, 3410, 2824, 3337, 2826, 3339, 2827, 2318, 3854, 272, 3342, 2834, 2835, 3

In [70]:
print(len(all_subgraph))
print(len(all_subgraph[1000]))
for i in range(len(all_subgraph)):
    if len(all_subgraph[i]) != 300:
        print("***")

1092
300


In [71]:
nodes_nums = {}
#子图数量可能会太多，删掉重复的
for subgraph in all_subgraph[::-1]:
    for n in subgraph:
        if n not in nodes_nums.keys():
            nodes_nums[n] = 1
        else:
            nodes_nums[n] = nodes_nums[n] + 1
if_del = []

# for subgraph in all_subgraph[::-1]:
#     flag = 1
#     for n in subgraph:
#         #不能删
#         if nodes_nums[n] <=1 :
#             flag = 0
#     if_del.append(flag)

In [72]:
print(if_del)
print(len(nodes_nums))

[]
3164


In [73]:
del_all_graph = copy.deepcopy(all_subgraph[::-1])
not_del = 0
for index,subgraph in enumerate(all_subgraph[::-1]):
    print(index)
    #遍历所有的子图，判断能不能删除，能删除就当场删除，并重新计算nodes_nums
    flag = 1
    for n in subgraph:
        #不能删
        if nodes_nums[n] <=1 :
            flag = 0  
            break
    if flag:
        del_all_graph.pop(not_del)
    else:
        not_del+=1
    # print(len(del_all_graph),'*')
    nodes_nums = {}
    #子图数量可能会太多，删掉重复的
    for subgraph in del_all_graph:
        for n in subgraph:
            if n not in nodes_nums.keys():
                nodes_nums[n] = 1
            else:
                nodes_nums[n] = nodes_nums[n] + 1


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [74]:
print(len(del_all_graph))
# print(del_all_graph)

158


In [75]:
nodes_nums = {}
#子图数量可能会太多，删掉重复的
for subgraph in del_all_graph:
    for n in subgraph:
        if n not in nodes_nums.keys():
            nodes_nums[n] = 1
        else:
            nodes_nums[n] = nodes_nums[n] + 1
print(nodes_nums)

{4096: 39, 1537: 74, 3585: 16, 4097: 24, 3077: 42, 1542: 39, 1543: 50, 2569: 14, 4106: 7, 3595: 26, 1548: 61, 1549: 26, 3086: 27, 4110: 2, 1552: 29, 1555: 22, 1556: 23, 1557: 21, 1558: 32, 1559: 40, 3093: 50, 3095: 13, 2582: 12, 2588: 12, 1565: 71, 1566: 41, 2078: 24, 2591: 27, 1569: 20, 1570: 13, 2592: 27, 1572: 28, 3108: 21, 3110: 6, 3622: 21, 1576: 47, 3621: 19, 3624: 29, 2042: 40, 1581: 42, 1582: 78, 1583: 40, 1584: 38, 4144: 4, 3634: 38, 4141: 40, 4148: 29, 1591: 39, 3128: 1, 2105: 87, 3641: 7, 1595: 70, 1596: 31, 3065: 13, 4155: 28, 2620: 30, 4160: 37, 2109: 39, 1602: 31, 1603: 5, 4164: 25, 1605: 35, 3133: 44, 1601: 50, 2631: 38, 3145: 34, 2635: 8, 3661: 15, 2638: 40, 2640: 37, 2129: 24, 1644: 50, 1619: 17, 2132: 14, 1621: 21, 3664: 57, 2647: 20, 2139: 11, 2140: 28, 3677: 28, 1630: 27, 2654: 34, 1632: 23, 3169: 47, 3167: 10, 2651: 52, 3175: 26, 104: 3, 1640: 64, 1642: 8, 1641: 73, 3691: 23, 2157: 18, 3693: 30, 3183: 18, 3184: 32, 3185: 47, 3698: 30, 2675: 14, 1652: 7, 1651: 40, 3

In [76]:
#再把节点数小于N的联通子图加上
# for c in init_graph:
#     c.extend([-1]*(N-len(c)))
# print(init_graph)

In [77]:
# del_all_graph.extend(init_graph)

In [78]:
# del_all_graph.pop(-1)
print(del_all_graph[len(del_all_graph)-1])

[1536, 1537, 3074, 4099, 1538, 3073, 2567, 8, 1544, 2059, 3599, 3093, 2070, 3096, 2040, 1562, 1563, 1564, 3099, 2074, 2591, 3615, 4127, 2082, 2083, 3108, 1573, 4128, 3618, 2079, 3621, 4129, 3118, 3631, 3632, 3633, 4143, 4148, 4150, 3639, 2555, 2105, 3131, 2619, 2110, 4159, 3646, 3648, 4158, 3651, 1605, 3142, 1607, 1609, 3657, 2123, 3659, 1612, 4171, 3663, 3664, 2638, 4170, 2132, 4181, 3672, 2650, 2140, 1629, 2143, 3680, 3169, 2144, 3171, 1636, 3175, 1640, 2664, 2154, 3691, 3689, 3181, 2667, 2668, 3696, 2673, 3698, 2675, 2163, 3185, 2678, 2165, 2168, 3194, 3710, 1663, 3712, 2177, 3716, 2693, 2181, 2692, 1673, 1674, 2697, 3212, 1679, 1681, 3730, 2198, 2199, 2712, 3737, 3738, 2201, 1691, 3739, 3743, 3744, 2210, 1700, 2213, 3238, 3752, 3245, 3759, 3248, 1714, 3764, 2229, 3254, 2231, 2744, 3256, 2745, 3769, 3262, 2752, 3777, 3266, 3776, 3269, 2759, 3274, 1738, 2771, 3286, 2078, 1763, 3300, 3812, 2278, 3304, 2794, 2796, 2284, 3313, 2802, 3314, 2592, 1777, 3320, 2809, 3323, 2300, 3325, 2812, 

In [79]:
import pandas as pd
data = pd.DataFrame(del_all_graph)
data.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,4096,1537,3585,4097,3077,1542,1543,2569,4106,3595,...,4085,1526,3575,1528,1529,1530,1531,1532,2036,4094
1,1536,3073,1537,1538,3588,1541,1542,2567,3079,1545,...,2036,1526,3575,1528,2041,1531,3068,3069,3070,1535
2,3585,4098,2051,3076,1543,1545,2569,3596,2062,3599,...,3055,3058,4084,2549,2038,4085,2040,1530,2045,1534
3,2560,1537,4096,2563,1540,2053,4102,2055,3591,1527,...,2548,4085,1526,3061,1528,1529,1530,2043,2558,1535
4,1537,4098,2561,2564,6,7,3080,3079,4106,3083,...,2544,2546,500,4084,1526,1528,3577,3067,1532,3581


In [80]:
print(len(del_all_graph))

158


In [81]:
data.to_csv("../data/SubGraph/N"+str(N)+"/N"+str(N)+"_"+str(idex)+"_"+str(len(del_all_graph))+".csv",index=False)

In [82]:
data = pd.read_csv("../data/SubGraph/N"+str(N)+"/N"+str(N)+"_"+str(idex)+"_"+str(len(del_all_graph))+".csv")
data.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,4096,1537,3585,4097,3077,1542,1543,2569,4106,3595,...,4085,1526,3575,1528,1529,1530,1531,1532,2036,4094
1,1536,3073,1537,1538,3588,1541,1542,2567,3079,1545,...,2036,1526,3575,1528,2041,1531,3068,3069,3070,1535
2,3585,4098,2051,3076,1543,1545,2569,3596,2062,3599,...,3055,3058,4084,2549,2038,4085,2040,1530,2045,1534
3,2560,1537,4096,2563,1540,2053,4102,2055,3591,1527,...,2548,4085,1526,3061,1528,1529,1530,2043,2558,1535
4,1537,4098,2561,2564,6,7,3080,3079,4106,3083,...,2544,2546,500,4084,1526,1528,3577,3067,1532,3581
